# Regressão Logística I

Índice

#### 1. [Análise descritiva](#1)  
1.1. [Bivariada](#1.1)  
1.2. [Medida de risco do evento](#1.2)  
1.3. [Chance de evento](#1.3)  
1.4. [Razão de chances (*odds ratio*)](#1.4)  
1.5. [Bivariada para variáveis contínuas](#1.5)  

#### 2. [Regressão logística](#2)  
2.1 [Função logística](#2.1)  
2.2 [Regressão logística com uma variável](#2.2)  
2.3 [Obtendo os parâmetros](#2.3)  
2.4 [Modelo com mais variáveis](#2.4)  
2.5 [Qualidade do modelo](#2.5)  

## 1. Introdução

Antes de rodar a regressão, vamos entender a análise básica sobre o problema, e alguns indicadores que sãomuito úteis para entender variáveis binárias, e vão ser muito úteis para entendermos a regressão logística.

Vamos entender a análise primeiramente com base em uma tabela de frequências em uma base já conhecida.

In [9]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### 1.1 Análise bivariada<a name="1.1"></a>
[Índice](#topo)  

A descritiva básica é baseada na tabela cruzada. Em seguida, vamos levantar alguns indicadores que nos ajudam a resumir e entender essa tabela, e se conectam com o modelo de regressão logística de uma forma muito interessante.

Vamos ilustrar com um exemplo, e a fórmula genérica será baseada na tabela abaixo:

| perfil | não evento      | evento      | total    |
|--------|----------|----------|----------|
| **P1** | $a_{11}$ | $a_{12}$ | $a_{1.}$ |
| **P2** | $a_{21}$ | $a_{22}$ | $a_{2.}$ |
| **Total** | $a_{.1}$ | $a_{.2}$ | $a_{..}$ |

In [3]:
titanic = sns.load_dataset('titanic')

In [4]:
tab = pd.crosstab(titanic['sex'], titanic['survived'], margins= True)
tab

### 1.2 Medida de risco do evento<a name="1.2"></a>
[Índice](#topo)  

O *risco* do evento é um sinônimo para a probabilidade de ocorrência de um evento, em geral quando esse evento tem uma conotação negativa (ex: inadimplência, fraude, morte etc).

No exemplo da nossa tabela, o "risco" sobreviver para passageiros do sexo feminino é:

$$P(sobreviver|feminino) = \dfrac{a_{12}}{a_{1.}} = \dfrac{233}{314} = 74\%$$

Observe que a probabilidade de **não sobreviver** (chamamos também não evento) pode ser escrita simplesmente como 1-P.

**Observação:**

- Quanto mais diferentes os riscos entre os perfis, maior o indício de relevância da variável para um modelo de classificação

In [ ]:
tab['probabilidade'] = tab[1]/tab['All']
tab

### 1.3 Chance (*Odds*)<a name="1.3"></a>
[Índice](#topo)  

Chance é uma medida de incerteza assim como probabilidade, mas de uma forma um pouco diferente. A chance de um evento é definida como o número de possibilidades para o evento dividido pelo número de possibilidades para o seu complementar. Por exemplo, a **chance** de tirar **6** em um dado é de **1 / 5** - dizemos **um para cinco**.

Na nossa tabela, a chance de **sobrevivência** para passageiros do sexo **feminino** pode ser calculada por:

$$Odds(feminino) = \dfrac{a_{12}}{a_{11}} = \dfrac{233}{81} = 2.9$$

Quando o evento é raro, o risco (ou probabilidade) do evento se aproximam numericamente do *odds*.

Vamos calcular na nossa tabela:

In [ ]:
tab['chance'] = tab[1]/tab[0]
tab

In [ ]:
2.87/0.23

### 1.4 Razão de chances (*odds ratio*)<a name="1.4"></a>
[Índice](#topo)  

A razão de chances é o que o nome diz: uma razão entre chances. Vamos entendê-la:

A razão de chances de **sobrevivênciai** de um passageiroo do sexo **feminino** com relação à chance de **sobrevivênciai** de um passageiro doo sexo **masculino** é de ```2,9/0,2 = 12,3```. Significa que uma **mulher** possui ```12.3``` vezes a chance de **sobrevivência** de homem.

Lembre-se disso: A razão de chances sempre compara as chances de um evento para dois públicos.

Na nossa tabela conceitual a fórmula fica assim:

$$ RC(P1/P2) = \dfrac{\dfrac{a_{12}}{a_{11}}}{{\dfrac{a_{22}}{a_{21}}}} = \dfrac{Odds(P1)}{Odds(P2)}$$

In [ ]:
tab.loc['male','chance']

In [ ]:
tab['RC_vs_male'] = tab['chance']/tab.loc['male','chance']
tab

#### Relação entre chance e probabilidade

Repare que a chance que escrevemos como:

$$chance = \dfrac{223}{81}$$

Também pode ser escrita assim:

$$chance = \dfrac{223}{81} = \dfrac{\dfrac{223}{314}}{\dfrac{81}{314}} = \dfrac{P(sobreviver|feminino)}{1-P(sobreviver|feminino)}$$

ou, de forma genérica:

$$chance = \dfrac{\dfrac{a_{12}}{a_{1.}}}{\dfrac{a_{11}}{a_{1.}}} = \dfrac{P(sem\_veículo)}{1-P(sem\_veículo)}$$

A chance pode também ser escrita assim:

$$Chance = \dfrac{P}{1-P} $$



## 2. Regressão logística<a name="2"></a>
[Índice](#topo)  

É a técnica estatística mais popular para modelagem preditiva de eventos binários. Assim como a regressão, é um modelo linear - o que significa que é definida por uma equação linear, ou seja uma equação da forma  

$g(y)=\alpha + \beta_1 x_1 + \beta_2 x_2 +...+ \beta_p x_p = \eta$  

Em que $\alpha, \beta_1, \beta_2,... \beta_p$ são constantes.

Portanto, a regressão logística compartilha uma série de características e ferramentas com os modelos de regressão linear, mas com particularidades para a resposta binária.

### 2.1 Função logística<a name="2.1"></a>
[Índice](#topo)  

O *logito* é a *função de ligação* que modelamos na regressão logística. É definido como o **logaritmo da chance**, ou ainda:

$$logito=log\left(\frac{p}{1-p}\right) $$

O que nos dá a seguinte equação:

$$log\left(\frac{p}{1-p}\right) = \alpha + \beta_1 x_1 + \beta_2 x_2 +...+ \beta_p x_p$$

- Quando P se aproxima de 0, o logito *vai para* $-\infty$.  
- Quando P se aproxima de 1, o logito *vai para* $+\infty$.  
- Não se define quando P=0 ou quando P=1.

Vamos observar isso graficamente:

In [ ]:
P = pd.Series(np.linspace(0.001,.999,1000))
P

In [ ]:
logito = np.log(P/(1-P))

plt.plot(P, logito, 'r');

#### Transformação inversa do logito

O nosso modelo linear vai nos fornecer uma estimativa do logito para cada perfil de interesse. Podemos *voltar* na probabilidade com a função inversa do logito. Com um pouco de álgebra, chegamos nessa equação aqui:


$$
    p = \frac{1}{\left(1+e^{-\eta}\right)}
$$


In [ ]:
plt.plot(logito, P, 'r')

### 2.2 Regressão logística com uma variável <a name="2.2"></a>
[Índice](#topo)  

A nossa regressão, como já falamos, é um modelo linear sobre uma função da probabilidade - essa função é o logito. Com uma variável, a equação da regressão logística que dá a probabilidade $P_i$ de default para cada cliente i=1, 2, ...N é:

$$ log\left(\dfrac{P_i}{\left(1-P_i\right)}\right) = \alpha + \beta x_i $$

Repare que se quisermos a chance para cada perfil, basta aplicar a função exponencial dos dois lados:

$$ \dfrac{P_i}{\left(1-P_i\right)} = e^{\alpha + \beta x_i} $$

Se quisermos as probabilidades, basta isolar o P na equação e obter:

$$ P_i = \dfrac{1}{1 + e^{- (\alpha + \beta x_i)}} $$



Vamos explorar a regressão logística com a variável do exemplo anterior para entender a equação.


In [10]:
import statsmodels.formula.api as smf

reglog = smf.logit("survived ~ sex",data=titanic).fit()
reglog.summary()

### 2.3 Obtendo as probabilidades estimadas<a name="2.3"></a>
[Índice](#topo)  

Além dos atributos que mencionamos e exploramos anteriormente, o objeto que armazena a regressão logística possui diversos métodos que fazem operações importantes como a aplicação da regra do modelo a uma base de dados. Essa base, no entanto, deve ter as mesmas variáveis nos mesmos nomes e formatos daquelas utilizadas na construção do modelo.

Assim, podemos calcular as probabilidades de evento para cada perfil "na mão" ou utilizando o método ```.predict()```.


In [ ]:
reglog.params[1]

In [ ]:
eta = reglog.params[0] + reglog.params[1]
eta

In [ ]:
chance = np.exp(eta)
chance

In [ ]:
p = 1/(1 + np.exp(-eta))
p

#### Escorando a base com .predict()

In [ ]:
titanic['predito'] = reglog.predict(titanic)

titanic

### 2.4 Modelo com mais variáveis<a name="2.4"></a>
[Índice](#topo)  

Vamos rodar um modelo mais completo nessa base. A estrutura é essencialmente a mesma, mas com mais variáveis.

$$ log\left(\dfrac{P_i}{1-P_i}\right) = \alpha + \beta_1 x_{i1}  + \beta_2 x_{i2} + ... + \beta_p x_{ip}$$

In [52]:
titanic.head()

In [75]:
reglog = smf.logit(" survived ~ sex + C(pclass) + fare"
                   , data=titanic).fit()

reglog.summary()

In [74]:
np.exp(-65*.0112)

In [58]:
eta = 2.2971 + (-2.6419) + (-1.9055)
chance = np.exp(eta)
prob = 1/(1+np.exp(-eta))
prob

In [60]:
1/np.exp(-2.6419)

In [65]:
np.exp(-1.9055)

### 2.5 Qualidade do modelo <a name="2.5"></a>
[Índice](#topo)  

Eixos de qualidade de um modelo:

- Calibragem: O quanto uma estimativa de probabilidade de evento esta próxima da realidade.
- Discriminação: O quanto o modelo consegue discriminar "eventos" de "não eventos" com probabilidades bem distintas.
- Estabilidade: A qualidade do modelo de ter pouca variabilidade temporal.
- Operacional: O quanto de pode confiar operacionalmente no modelo (consistência dos dados, infra-estrutura confiável, etc).

In [142]:
titanic['predito'] = reglog.predict(titanic)
titanic.head()

#### Calibragem

In [144]:
cat_pred = pd.qcut(titanic['predito'], 5, duplicates='drop')
cat_pred.value_counts().sort_index()

In [145]:
group_reg = titanic.groupby(cat_pred)
qualid = group_reg[['survived']].count().rename(columns = {'survived': 'contagem'})
qualid

In [146]:
qualid['predito'] = group_reg['predito'].mean()
qualid

In [147]:
qualid['pct_sobrev'] = group_reg['survived'].mean()
qualid

In [148]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax = qualid['pct_sobrev'].plot(label='%Predito')
ax = qualid['predito'].plot(label='%Observado')

ticks = ax.set_xticks([0, 1, 2, 3, 4])
labels = ax.set_xticklabels([1, 2, 3, 4, 5])
ax.legend(loc="lower right")
ax.set_ylabel('Probabilidade de evento')
ax.set_xlabel('Grupo')


#### Discriminação

Curva ROC

In [150]:
from sklearn.metrics import roc_curve
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(titanic['survived'], titanic['predito'])

plt.figure()
lw = 2

fpr, tpr, thresholds = metrics.roc_curve(titanic['survived'], titanic['predito'])
auc_ = metrics.auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % auc_)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()



Gráfico do KS

In [154]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

score_pop1 = titanic.loc[titanic['survived']==1,'predito']
ax.plot(np.sort(score_pop1), np.linspace(0, 1, len(score_pop1), endpoint=False), label='Sobreviventes')

score_pop2 = titanic.loc[titanic['survived']!=1,'predito']
ax.plot(np.sort(score_pop2), np.linspace(0, 1, len(score_pop2), endpoint=False), label='Não sobreviventes')
ax.legend()

ax.set_xlabel('P')
ax.set_ylabel('Função Distribuição Acumulada')


#### Indicadores

In [155]:
from sklearn import metrics
from scipy.stats import ks_2samp

acc = metrics.accuracy_score(titanic['survived'], titanic['predito']>.5)
print('Acurácia: {0:.2f}%'.format(acc*100))

#AUC
fpr, tpr, thresholds = metrics.roc_curve(titanic['survived'], titanic['predito'])
auc_ = metrics.auc(fpr, tpr)
#Gini
gini = 2*auc_ -1
#KS
ks = ks_2samp(titanic.loc[titanic['survived'] == 1, 'predito'], titanic.loc[titanic['survived'] != 1, 'predito']).statistic

print('KS: {0:.2f}% \nAUC: {1:.2f}% \nGINI: {2:.2f}%'
      .format(ks*100, auc_*100, gini*100))